In [1]:
import os
import re
import json
from datetime import datetime

In [2]:
class email:
    def __init__(self, id, message_id, data, subject, sender, receiver, day_week, time, time_zone, date, beginning, farewell, answer, length, relative_path):
        self._id = id
        self._message_id = message_id
        self._data = data
        
        # Verificar si 'subject' es una lista y convertirlo a cadena si es necesario
        if isinstance(subject, list):
            subject = " ".join(subject)
        
        self._subject = subject
        self._sender = sender
        self._receiver = receiver
        self._number_of_receivers = len(receiver)
        self._day_week = day_week
        self._time = time
        self._time_zone = time_zone
        self._date = date
        self._responded = False
        self._responses = set()
        self._beginning = beginning
        self._farewell = farewell
        self._answer = answer
        self._length = length

        # Determinar si el correo es una respuesta y almacenar el resultado
        self._is_reply = self._subject.startswith("Re: ")
        
        self._relative_path = relative_path

# Luego, al crear una instancia de 'email', esta lógica se ejecutará automáticamente


In [3]:
def email_to_dict(email):
    return email.__dict__

In [4]:
def read_email(filename, message_ids, id):
    with open(filename, 'r', errors='ignore') as file:
        data = file.read()
        id = id
        Date = r'Date: (\w+), (\d+ \w+ \d+) (\d+:\d+:\d+) (-\d{4} \(.*?\))'
        coincidencias = re.findall(Date, data)
        for day_week, date, time, time_zone in coincidencias:
            day_week = day_week
            date = date
            time = time
            time_zone = time_zone
            
        subject = re.findall(r'Subject: (.*)',data)
        
        message_id = re.search(r'Message-ID: (.*)',data)
        if message_id:
            message_id = message_id.group(1)
            
        sender_match = re.search(r'\nFrom: (.*)', data)
        if sender_match:
            sender = sender_match.group(1)
        else:
            sender = None
        
        receiver = re.findall(r'\nTo: (.*)',data)
        input_data = str(receiver)[1:-1]
        receiver_list = input_data.split(', ')
        receiver_list = [receiver.strip() for receiver in receiver_list]
        receiver_list = [s.replace("'", "") for s in receiver_list]
        receiver = [s.replace("\"", "'") for s in receiver_list]
        
        
        beginning_pattern = (r"(?:\b(?:hello|hi|hey|good (?:morning|afternoon|evening)|greetings|dear|my dearest|hi there|hello there|hi all|hi everyone|hello everyone|dearest|yo|what's up|hey pal|yolo|howdy|how's it going|how are you|what's happening|what's new|good to see you|long time no see|it's been a while|what's cracking|how's everything))\b")
        beginning = re.findall(beginning_pattern, data, flags=re.IGNORECASE)
        
        
        farewell_pattern = r"(?:\b(?:Sincerely|Regards|Best regards|Kind regards|Yours sincerely|Yours faithfully|Best wishes|Take care|Warm regards|With gratitude|With best regards|Cheers|Cordially|Respectfully|Until next time|All the best|Have a great day|Talk to you soon|Looking forward to hearing from you|With thanks|Later|Catch you later|Farewell|Peace out|Stay well|All my best|Goodbye|ByeFond regards|Take it easy|So long|See you around|Keep in touch|With warm regards|With all my heart)\b)"
        farewell = re.findall(farewell_pattern, data, flags=re.IGNORECASE)
        
        if re.search(r"-----Original Message-----", data):
            answer = 1
        elif subject[0].startswith("Re:"):
            answer = 1
        else:
            answer = 0
        
        
        body_b = data.find("X-FileName:")
        if body_b != -1:
            end_l = data.find("\n", body_b)
            if end_l != -1:
                body = data[end_l + 1:]
                words = body.split()
        length = len(words)

        relative_path = os.path.relpath(filename, start=os.path.join(os.getcwd(), '..'))
        
        if message_id in message_ids:
            return None
        else:
            email_object = email(id, message_id, data, subject, sender, receiver, day_week,time, time_zone, date,beginning,farewell, answer, length, relative_path)
            return email_object
        

In [5]:
list_of_emails = []
set_of_message_ids = set()
counter = 0


# base_dir = "../maildir"
base_dir = os.path.join(os.path.dirname(os.getcwd()), 'maildir')


for root, dirs, files in os.walk(base_dir):
    for file in files:
        file_path = os.path.join(root, file)
        tmp_email = read_email(file_path, set_of_message_ids, counter)
        counter += 1
        if tmp_email is not None:
            list_of_emails.append(tmp_email)
            set_of_message_ids.add(tmp_email._message_id)

In [6]:
# Contar cuántos correos son respuestas, teniendo en cuenta que '_subject' podría ser una lista
count_replies = 0
for email in list_of_emails:
    subject = email._subject
    # Si '_subject' es una lista, unimos sus elementos en una cadena
    if isinstance(subject, list):
        subject = " ".join(subject)
    # Verificar si el asunto resultante contiene 'Re:'
    if subject.startswith("Re: "):
        count_replies += 1

# Imprimir el total de correos que son respuestas
print(f'Total de correos que son respuestas: {count_replies}')

Total de correos que son respuestas: 645


In [7]:
def find_parent_mail(_id, _subject, _receiver, _date, _time):
    _subject = re.sub(r'Re: ', '', _subject)
    _subject = _subject.replace(' ', '')
    for email in list_of_emails:
        email_datetime_str = email._date + ' ' + email._time
        email_datetime = datetime.strptime(email_datetime_str, '%d %b %Y %H:%M:%S')
        _datetime_str = _date + ' ' + _time
        _datetime = datetime.strptime(_datetime_str, '%d %b %Y %H:%M:%S')
        if email._subject.replace(' ', '').replace('Re:', '') == _subject.replace(' ', '') and email._sender in _receiver and email_datetime < _datetime:
            email._responded = True
            email._responses.add(_id)

In [8]:
for email in list_of_emails:
    if email._is_reply:
        id = email._id
        subject = email._subject
        receiver = email._receiver
        date = email._date
        time = email._time
        find_parent_mail(id, subject, receiver, date, time)

In [9]:
for email in list_of_emails:
    email._responses = list(email._responses)

In [10]:
for counter in range(1, len(list_of_emails), 100000):
    emails_dict_list = [email_to_dict(email) for email in list_of_emails[(counter - 1):(counter + 99999)]]
    file_path = f'../data/emails{(counter // 100000) + 1}.json'
    with open(file_path, 'w') as json_file:
        json.dump(emails_dict_list, json_file)